### Import Libraries

In [4]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras import regularizers
import os
import numpy as np
import pandas as pd

### View FER-2013 Dataset

In [8]:
train_dir = "./emotions/train/"
test_dir = "./emotions/test/"

row, col = 48, 48
classes = 3

def count_data(path, set_):
    dict_ = {}
    for emotion in os.listdir(path):
        if not emotion.startswith('.') and emotion != 'sad':
            dir_ = path + emotion
            dict_[emotion] = len(os.listdir(dir_))
    df = pd.DataFrame(dict_, index = [set_])
    return df

train_count = count_data(train_dir, 'train')
test_count = count_data(test_dir, 'test')
print(train_count)
print(test_count)

       angry  happy  neutral
train   3995   7215     4965
      angry  happy  neutral
test    958   1774     1233


### Create Training and Test Sets

In [9]:
train_gen = ImageDataGenerator(rescale = 1. / 255, zoom_range = 0.3, horizontal_flip = True)
train_set = train_gen.flow_from_directory(train_dir,
                                          classes = ['angry', 'happy', 'neutral'],
                                          batch_size = 64, 
                                          target_size = (row, col),
                                          shuffle = True,
                                          color_mode = 'grayscale', 
                                          class_mode = 'categorical')

test_gen = ImageDataGenerator(rescale = 1. / 255)
test_set = test_gen.flow_from_directory(test_dir, 
                                        classes = ['angry', 'happy', 'neutral'],
                                        batch_size = 64,
                                        target_size = (row, col), 
                                        shuffle = True, 
                                        color_mode = 'grayscale', 
                                        class_mode = 'categorical')

print(train_set.class_indices)
print(test_set.class_indices)

Found 16175 images belonging to 3 classes.
Found 3965 images belonging to 3 classes.
{'angry': 0, 'happy': 1, 'neutral': 2}
{'angry': 0, 'happy': 1, 'neutral': 2}


### Create the CNN

In [19]:
MODEL_NAME = 'emotions_v4.hd5'

def build_model(model_name, input_size, classes = 3):
    if os.path.exists(model_name):
        model = load_model(model_name)
    else:      
        model = Sequential()
        
        model.add(Conv2D(32, kernel_size = (3, 3), padding = 'same', activation = 'relu', input_shape = input_size))
        model.add(BatchNormalization())
        model.add(Conv2D(32, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size = (2, 2), strides = 2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(64, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
        model.add(BatchNormalization())
        model.add(Conv2D(64, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size = (2, 2), strides = 2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(128, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
        model.add(BatchNormalization())
        model.add(Conv2D(128, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size = (2, 2), strides = 2))
        model.add(Dropout(0.25))
        
        model.add(Flatten())
        
        model.add(Dense(1024, activation = 'relu'))
        model.add(Dropout(0.25))
        model.add(Dense(512, activation = 'relu'))
        model.add(Dropout(0.25))
        model.add(Dense(256, activation = 'relu'))
        model.add(Dropout(0.25))
        model.add(Dense(classes, activation = 'softmax'))
    
    return model

model = build_model(MODEL_NAME, (row, col, 1), classes)
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_54 (Conv2D)          (None, 48, 48, 32)        320       
                                                                 
 batch_normalization_54 (Bat  (None, 48, 48, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_55 (Conv2D)          (None, 48, 48, 32)        9248      
                                                                 
 batch_normalization_55 (Bat  (None, 48, 48, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 24, 24, 32)       0         
 g2D)                                                            
                                                      

### Training the model

In [23]:
def train(model, train_set, test_set, epochs, model_name):
    model.compile(optimizer = Adam(learning_rate = 0.0001, decay = 1e-6), 
                  loss = 'categorical_crossentropy', 
                  metrics = ['accuracy'])
    
    savemodel = ModelCheckpoint(filepath = model_name,
                                save_best_only = True,
                                verbose = 1,
                                mode = 'min',
                                monitor = 'val_loss')
    stopmodel = EarlyStopping(monitor = 'val_loss',
                              min_delta = 0.001, 
                              patience = 3,
                              verbose = 1,
                              restore_best_weights = True)
    reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', 
                                  factor = 0.2, 
                                  patience = 6, 
                                  verbose = 1, 
                                  min_delta = 0.001)
    
    steps_per_epoch = train_set.n // train_set.batch_size
    validation_steps = test_set.n // test_set.batch_size
    
    print("Starting training.")
    
    model.fit(x = train_set, 
              validation_data = test_set, 
              epochs = epochs, 
              callbacks = [savemodel, stopmodel, reduce_lr],
              steps_per_epoch = steps_per_epoch, 
              validation_steps = validation_steps)
    
    print("Done. Now evaluating.")
    loss, acc = model.evaluate(test_set)
    print("Test accuracy: %3.2f, loss: %3.2f" % (acc, loss))
    
train(model, train_set, test_set, 10, MODEL_NAME)

Starting training.
Epoch 1/10
252/252 [==============================] - ETA: 0s - loss: 0.5080 - accuracy: 0.7913
Epoch 1: val_loss improved from inf to 0.48446, saving model to emotions_v4.hd5


INFO:tensorflow:Assets written to: emotions_v4.hd5/assets


INFO:tensorflow:Assets written to: emotions_v4.hd5/assets


252/252 [==============================] - 92s 361ms/step - loss: 0.5080 - accuracy: 0.7913 - val_loss: 0.4845 - val_accuracy: 0.8040 - lr: 1.0000e-04
Epoch 2/10
252/252 [==============================] - ETA: 0s - loss: 0.5043 - accuracy: 0.7937
Epoch 2: val_loss did not improve from 0.48446
252/252 [==============================] - 90s 356ms/step - loss: 0.5043 - accuracy: 0.7937 - val_loss: 0.4948 - val_accuracy: 0.8064 - lr: 1.0000e-04
Epoch 3/10
252/252 [==============================] - ETA: 0s - loss: 0.4921 - accuracy: 0.7950
Epoch 3: val_loss improved from 0.48446 to 0.47704, saving model to emotions_v4.hd5


INFO:tensorflow:Assets written to: emotions_v4.hd5/assets


INFO:tensorflow:Assets written to: emotions_v4.hd5/assets


252/252 [==============================] - 90s 356ms/step - loss: 0.4921 - accuracy: 0.7950 - val_loss: 0.4770 - val_accuracy: 0.8071 - lr: 1.0000e-04
Epoch 4/10
252/252 [==============================] - ETA: 0s - loss: 0.4914 - accuracy: 0.8005
Epoch 4: val_loss did not improve from 0.47704
252/252 [==============================] - 88s 350ms/step - loss: 0.4914 - accuracy: 0.8005 - val_loss: 0.4925 - val_accuracy: 0.8117 - lr: 1.0000e-04
Epoch 5/10
252/252 [==============================] - ETA: 0s - loss: 0.4824 - accuracy: 0.8029
Epoch 5: val_loss did not improve from 0.47704
252/252 [==============================] - 91s 361ms/step - loss: 0.4824 - accuracy: 0.8029 - val_loss: 0.5079 - val_accuracy: 0.8028 - lr: 1.0000e-04
Epoch 6/10
252/252 [==============================] - ETA: 0s - loss: 0.4785 - accuracy: 0.8047
Epoch 6: val_loss improved from 0.47704 to 0.47571, saving model to emotions_v4.hd5


INFO:tensorflow:Assets written to: emotions_v4.hd5/assets


INFO:tensorflow:Assets written to: emotions_v4.hd5/assets


252/252 [==============================] - 89s 352ms/step - loss: 0.4785 - accuracy: 0.8047 - val_loss: 0.4757 - val_accuracy: 0.8117 - lr: 1.0000e-04
Epoch 7/10
252/252 [==============================] - ETA: 0s - loss: 0.4766 - accuracy: 0.8057
Epoch 7: val_loss did not improve from 0.47571
252/252 [==============================] - 92s 365ms/step - loss: 0.4766 - accuracy: 0.8057 - val_loss: 0.5067 - val_accuracy: 0.8066 - lr: 1.0000e-04
Epoch 8/10
252/252 [==============================] - ETA: 0s - loss: 0.4763 - accuracy: 0.8056
Epoch 8: val_loss did not improve from 0.47571
252/252 [==============================] - 90s 357ms/step - loss: 0.4763 - accuracy: 0.8056 - val_loss: 0.4875 - val_accuracy: 0.8066 - lr: 1.0000e-04
Epoch 9/10
252/252 [==============================] - ETA: 0s - loss: 0.4667 - accuracy: 0.8098
Epoch 9: val_loss did not improve from 0.47571
Restoring model weights from the end of the best epoch: 6.
252/252 [==============================] - 91s 361ms/step -